In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).


In [ ]:
import pandas as pd
my_sales_train = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/sales_train.csv")

my_sales_train["date"] = pd.to_datetime(my_sales_train["date"])

answer = my_sales_train.groupby("item_id").sum().sort_values(by = 'item_cnt_day', ascending = False).reset_index().head(10).rename(index = {0:1, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10 })
print("Топ часто покупаемых товаров: ")
display(answer["item_id"])

Топ часто покупаемых товаров: 


1     20949
2      2808
3      3732
4     17717
5      5822
6      3734
7      6675
8      3731
9      1855
10    16787
Name: item_id, dtype: int64

2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
answer_2013_year = my_sales_train.loc[my_sales_train["date"].dt.year == 2013]
answer_2014_year = my_sales_train.loc[my_sales_train["date"].dt.year == 2014]
answer_2015_year = my_sales_train.loc[my_sales_train["date"].dt.year == 2015]
answer_all = pd.concat([answer_2013_year, answer_2014_year, answer_2015_year]) # по умолчанию axis = 0
answer_2 = answer_all.groupby("item_id").sum().sort_values(by = 'item_cnt_day', ascending = False).reset_index().head(10).rename(index = {0:1, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10 })
print("Топ проданных товаров, покупаемых в 2013, 2014, 2015")
display(answer_2["item_id"])

Топ проданных товаров, покупаемых в 2013, 2014, 2015


1     20949
2      2808
3      3732
4     17717
5      5822
6      3734
7      6675
8      3731
9      1855
10    16787
Name: item_id, dtype: int64

# 2 Задание 

1.   Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2.   Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.





In [ ]:
my_sales_train = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/sales_train.csv")
category_frame = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/item_categories.csv")
items_frame = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/items.csv")
my_sales_train = my_sales_train[["item_cnt_day", "item_id"]]
train_items = items_frame.set_index('item_id').join(my_sales_train.set_index('item_id')).reset_index()


# Сгруппируем сразу весь сет, чтобы потом было просто выделить топ и боттом
sorted_values = train_items.groupby('category_id').sum().sort_values(by = "item_cnt_day", ascending = False).reset_index()

# Заметим, что на последнем месте находятся 2 категории товаров
united = sorted_values.join(category_frame.set_index("item_category_id"), on='category_id')
print("Топ")
display(united[["category_id", "item_category_name"]].head(1))
print("Низ")
display(united[["category_id", "item_category_name"]].tail(2))





  

Топ


,category_id,item_category_name
0,40,Cinema - DVD


Низ


,category_id,item_category_name
82,10,Game consoles - PS2
83,51,Books - Cognitive literature


# 3 Задание


1. Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими заказами



In [ ]:
# Воспользуемся двумя таблицами. Items для связи item_id и category_id, а также sales_train для связи shop_id с item_id и item_cnt_day
my_sales_train = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/sales_train.csv")
items_frame = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/items.csv")
train_items = items_frame.join(my_sales_train.set_index("item_id"), on="item_id").reset_index()
train_items.pop("index")

# Создадим новый датафрейм для товаров из категории 40
new_df = train_items[train_items["category_id"] == 40]
#display(new_df.sample(10))
# Теперь найдем только те магазины топ 5, которые продавали товары категории 40 
all_avg = new_df.groupby(["shop_id"]).sum().sort_values(by = "item_cnt_day", ascending = False).reset_index().head(5)
#display(all_avg)
# Магазины с наибольшими продажами товаров категории 40 = 31, 25, 54, 57, 28
# Найдем средние продажи товаров в этих 5 магазинах категории 
# Снизу приведен список магазинов (топ 5), которые продали больше всего товаров категории 40
n_df = new_df[(new_df["shop_id"] == 31) | (new_df["shop_id"] == 25) | (new_df["shop_id"] == 54) | (new_df["shop_id"] == 57) | (new_df["shop_id"] == 28)]

# Найдем средние продажи каждого товара категории 40 из топ 5 магазинов по продажам этой категории
avg_df = n_df.groupby('item_id').mean().sort_values(by = 'item_cnt_day', ascending = False).reset_index().head(10).rename(columns = {"item_cnt_day": "avg_cnt_day"})
display(avg_df[["item_id", "avg_cnt_day", "category_id"]])

,item_id,avg_cnt_day,category_id
0,10447,4.875000,40.0
1,19655,4.500000,40.0
2,21449,4.250000,40.0
3,16581,2.989051,40.0
4,21376,2.927900,40.0
5,13811,2.680101,40.0
6,14574,2.441509,40.0
7,16482,2.411765,40.0
8,10956,2.335463,40.0
9,19811,2.306748,40.0


3.2 Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.

In [ ]:
# Для каждого товара сверху выделим месяц продаж (date в my_sales_train)
sales_train = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/sales_train.csv")
items_frame = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/items.csv")
# Преобразуем в datetime
sales_train["date"] = pd.to_datetime(sales_train["date"])
train_items = items_frame.join(sales_train.set_index("item_id"), on="item_id").reset_index()
display(train_items.head(5))
train_items.insert(1, "month", train_items["date"].dt.month.astype("Int32"))
train_items.insert(1, "year", train_items["date"].dt.year.astype("Int32"))
max_sales = train_items.groupby(['item_id', 'month']).agg({'item_cnt_day': 'sum'}).sort_values(by = ['item_id', 'item_cnt_day'], ascending = [True, False]).reset_index()
num_max_sales = max_sales.drop_duplicates(["item_id"], keep = 'first')
num_min_sales = max_sales.drop_duplicates(["item_id"], keep = 'last')
display(num_max_sales.head(3))
display(num_min_sales.head(5))

,index,item_name,item_id,category_id,date,date_block_num,shop_id,item_price,item_cnt_day
0,0,!! IN THE POWER OF HAPPINESS (PLAST) D,0,40,2014-01-09,20.0,54.0,58.0,1.0
1,1,! ABBYY FineReader 12 Professional Edition Ful...,1,76,2014-04-04,15.0,55.0,4490.0,1.0
2,1,! ABBYY FineReader 12 Professional Edition Ful...,1,76,2014-02-04,15.0,55.0,4490.0,1.0
3,1,! ABBYY FineReader 12 Professional Edition Ful...,1,76,2014-06-07,18.0,55.0,4490.0,1.0
4,1,! ABBYY FineReader 12 Professional Edition Ful...,1,76,2014-04-08,19.0,55.0,4490.0,1.0


,item_id,month,item_cnt_day
0,0,1,1.0
1,1,4,2.0
6,2,8,1.0


,item_id,month,item_cnt_day
0,0,1,1.0
5,1,11,1.0
7,2,12,1.0
9,3,8,1.0
10,4,9,1.0


# Задание 4
1. Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце



In [ ]:
my_sales_train = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/sales_train.csv")
items_frame = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/items.csv")
sales_train["date"] = pd.to_datetime(sales_train["date"])
train_items = items_frame.join(sales_train.set_index("item_id"), on="item_id").reset_index()
train_items.pop("index")
train_items = train_items[['item_name', "category_id", "date", "item_id", "item_price"]]
# Изымем месяц и добавим его в начало df в int виде (По умолчанию добавляет 8.0, 9.0 и прочее)
train_items.insert(1, "month", train_items["date"].dt.month.astype("Int32"))

# Отсортируем по категориям и отсотрированным по месяцам среднюю и медиану
train_items = train_items.groupby(['category_id', 'month']).agg({'item_price': ['mean', 'median']}).reset_index()
display(train_items.head(5))

category_id month   item_price        
                            mean  median
0           0     2    57.700000    57.7
1           0    10   148.000000   148.0
2           1     2   283.000000   283.0
3           1    10   148.000000   148.0
4           2     1  2229.932607  2390.0

In [ ]:
# Найдем в каждом году месяцы, в которых средняя цена ниже, чем в предыдущем и следующем
my_sales_train = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/sales_train.csv")
items_frame = pd.read_csv("/content/drive/MyDrive/ВИМ/1ВИМ/items.csv")
sales_train["date"] = pd.to_datetime(sales_train["date"])
train_items = items_frame.join(sales_train.set_index("item_id"), on="item_id").reset_index()
train_items.pop("index")
train_items = train_items[['item_name', "category_id", "date", "item_id", "item_price"]]
# Выразим в таблице номер года и месяца 
train_items.insert(1, "month", train_items["date"].dt.month.astype("Int32"))
train_items.insert(1, "year", train_items["date"].dt.year.astype("Int32"))

# Найдем те месяцы, в которых средняя цена ниже, чем в двух соседних
train_items = train_items.groupby(['category_id', 'year', 'month']).agg({'item_price': ['median', 'mean']}).reset_index()
display(train_items.head(10))

# Начнем находить нужные месяцы (если значение не меньше, то идем на 2 шага вниз). PS с apply не понял, как сделать не изолированный подсчет с соседями :(
for i in range(1, len(train_items.index) - 1):
    if (train_items.iloc[i, -1] < train_items.iloc[i+1, -1]) and (train_items.iloc[i, -1] < train_items.iloc[i-1, -1]):
      print(train_items.loc[i, ['category_id', 'year', 'month']].to_string() + '\n')
      # список месяцев, в которых средняя цена данной категории товара была ниже, чем два соседних.


    

category_id  year month item_price             
                              median         mean
0           0  2013     2       57.7    57.700000
1           0  2013    10      148.0   148.000000
2           1  2013     2      283.0   283.000000
3           1  2013    10      148.0   148.000000
4           2  2013     1     2390.0  2206.875640
5           2  2013     2     2390.0  2080.364910
6           2  2013     3     2390.0  2189.764962
7           2  2013     4     2390.0  2196.349645
8           2  2013     5     2290.0  2128.431692
9           2  2013     6     2390.0  2225.294741

category_id         1
year             2013
month              10

category_id         2
year             2013
month               2

category_id         2
year             2013
month               5

category_id         2
year             2013
month               7

category_id         2
year             2013
month              12

category_id         2
year             2014
month               4

category_id         2
year             2014
month               6

category_id         2
year             2014
month              10

category_id         2
year             2015
month               2

category_id         2
year             2015
month               5

category_id         2
year             2015
month               8

category_id         3
year             2013
month               2

category_id         3
year             2013
month               4

category_id         3
year             2013
month               9

category_id         3
year             2014
month             

## R домашняя работа
https://colab.research.google.com/drive/1WU8iB8njt8fRRDYBXZIAzI7bpTqdZ1B9